## 라이브러리 & 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import gc
import random
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import Adam

import re
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = 'data/'

In [12]:
train_err = pd.read_csv(PATH+'train_err_data.csv')
train_qua = pd.read_csv(PATH+'train_quality_data.csv', thousands = ',')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')

train_qua = train_qua.drop(['time', 'fwver', 'quality_3', 'quality_4'], axis=1)
train_qua = train_qua.drop_duplicates()
train_qua = train_qua.reset_index(drop=True)

test_err = pd.read_csv(PATH+'test_err_data.csv')
test_qua = pd.read_csv(PATH+'test_quality_data.csv', thousands = ',')

test_qua = test_qua.drop(['time', 'fwver', 'quality_3', 'quality_4'], axis=1)
test_qua = test_qua.drop_duplicates()
test_qua = test_qua.reset_index(drop=True)

In [13]:
# error 데이터는 quality 데이터와 겹치는 user id가 있고 없음으로 나누었다.
train_err1 = train_err[train_err['user_id'].isin(train_qua['user_id'].unique())==False]
train_err2 = train_err[train_err['user_id'].isin(train_qua['user_id'].unique())]

test_err1 = test_err[test_err['user_id'].isin(test_qua['user_id'].unique())==False]
test_err2 = test_err[test_err['user_id'].isin(test_qua['user_id'].unique())]

train_err1_len = len(train_err1['user_id'].unique())
train_err2_len = len(train_err2['user_id'].unique())
print("train_err1에서 user_id의 개수는 ", train_err1_len)
print("train_err2에서 user_id의 개수는 ", train_err2_len)

# train_qua.columns = [n for n in range(12)]
# test_qua.columns = [n for n in range(12)]

train_err1에서 user_id의 개수는  6719
train_err2에서 user_id의 개수는  8281


## errtype

In [4]:
# train_err1 데이터화
id_error = train_err[['user_id','errtype']].values
error_type = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
    error_type[person_idx-10000, err-1] += 1
error_type.shape

  0%|          | 0/16554663 [00:00<?, ?it/s]

(15000, 42)

In [5]:
id_error = test_err[['user_id','errtype']].values
error_type_test = np.zeros((14999,42))

for person_idx, err in tqdm(id_error):
    error_type_test[person_idx-30000, err-1] += 1
error_type_test.shape

  0%|          | 0/16532648 [00:00<?, ?it/s]

(14999, 42)

## errcode

In [6]:
err_space = 120
temp = list(train_err['errcode'].value_counts()[:err_space].index)
train = train_err[train_err['errcode'].isin(temp)]

encoder_errcode = LabelEncoder()
encoder_errcode.fit(sorted(list(train['errcode'].unique())))

train['errcode'] = encoder_errcode.transform(train['errcode'])

In [7]:
id_error = train[['user_id','errcode']].values
error_code = np.zeros((15000, err_space))

for person_idx, err in tqdm(id_error):
    error_code[person_idx-10000, err] += 1

error_code.shape

  0%|          | 0/16550119 [00:00<?, ?it/s]

(15000, 120)

In [8]:
test = test_err[test_err['errcode'].isin(temp)]
test['errcode'] = encoder_errcode.transform(test['errcode'])

In [9]:
id_error = test[['user_id','errcode']].values
error_code_test = np.zeros((14999, err_space))

for person_idx, err in tqdm(id_error):
    error_code_test[person_idx-30000, err] += 1

error_code_test.shape

  0%|          | 0/16527246 [00:00<?, ?it/s]

(14999, 120)

## quality

In [16]:
quality = np.zeros((15000,2612,9))

for idx in tqdm(train_qua['user_id'].unique()):
    quality[idx-10000] = np.vstack((np.array(train_qua[train_qua['user_id']==idx].iloc[:, 3:12]), np.zeros((2612-train_qua[train_qua['user_id']==idx].shape[0],9))))

quality.shape

  0%|          | 0/8281 [00:00<?, ?it/s]

(15000, 2612, 9)

In [19]:
# quality_list = list(train_qua.columns)
# del quality_list[quality_list.index('time')]
# del quality_list[quality_list.index('fwver')]

# id_quality = train_qua[quality_list].values
# quality_type = np.zeros((15000,11))

# for person_idx in tqdm(id_quality):
#     quality_type[person_idx-10000, err-1] += 1
# quality_type.shape

In [20]:
quality_ui = np.zeros(15000)
quality_ui[train_qua.user_id.unique()-10000] = 1
quality_ui = quality_ui.reshape(-1,1)

In [21]:
quality_str = np.hstack([np.max(quality, axis=1) - np.mean(quality, axis=1),
                         np.mean(quality, axis=1) - np.min(quality, axis=1),
                         np.mean(quality, axis=1), quality_ui])
quality_str.shape

(15000, 28)

In [22]:
# quality_type = np.zeros((15000,11))

# for user_id in tqdm(range(10000, 25000)):
#     for quality in range(1, 12):
#         try:
#             quality_type[user_id-10000, quality-1] = (train_qua[train_qua[0]==user_id][quality]==(-1)).value_counts()[1]
#         except:
#             quality_type[user_id-10000, quality-1] = 0

# quality_type.shape

In [23]:
quality_test = np.zeros((14999,2612,9))

for idx in tqdm(test_qua['user_id'].unique()):
    quality_test[idx-30000] = np.vstack((np.array(test_qua[test_qua['user_id']==idx].iloc[:, 3:12]), np.zeros((2612-test_qua[test_qua['user_id']==idx].shape[0],9))))

quality_test.shape

  0%|          | 0/8268 [00:00<?, ?it/s]

(14999, 2612, 9)

In [24]:
quality_ui_test = np.zeros(14999)
quality_ui_test[test_qua.user_id.unique()-30000] = 1
quality_ui_test = quality_ui_test.reshape(-1,1)

In [25]:
quality_str_test = np.hstack([np.max(quality_test, axis=1) - np.mean(quality_test, axis=1),
                              np.mean(quality_test, axis=1) - np.min(quality_test, axis=1),
                              np.mean(quality_test, axis=1), quality_ui_test])
quality_str_test.shape

(14999, 28)

In [26]:
# quality_type_test = np.zeros((14999,11))

# for user_id in tqdm(range(30000, 44999)):
#     for quality in range(1, 12):
#         try:
#             quality_type_test[user_id-30000, quality-1] = (test_qua[test_qua[0]==user_id][quality]==(-1)).value_counts()[1]
#         except:
#             quality_type_test[user_id-30000, quality-1] = 0

# quality_type_test.shape

## problem

In [27]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

## 최종 데이터 정리

In [28]:
error_type1 = error_type[train_err1['user_id'].unique()-np.array(10000)]
error_type2 = error_type[train_err2['user_id'].unique()-np.array(10000)]

error_code1 = error_code[train_err1['user_id'].unique()-np.array(10000)]
error_code2 = error_code[train_err2['user_id'].unique()-np.array(10000)]

quality_str = quality_str[train_err2['user_id'].unique()-np.array(10000)]

problem1 = problem[train_err1['user_id'].unique()-np.array(10000)]
problem2 = problem[train_err2['user_id'].unique()-np.array(10000)]

error_type_test1 = error_type_test[test_err1['user_id'].unique()-np.array(30000)]
error_type_test2 = error_type_test[test_err2['user_id'].unique()-np.array(30000)]

error_code_test1 = error_code_test[test_err1['user_id'].unique()-np.array(30000)]
error_code_test2 = error_code_test[test_err2['user_id'].unique()-np.array(30000)]

quality_str_test = quality_str_test[test_err2['user_id'].unique()-np.array(30000)]

In [32]:
train_for_one = np.hstack([error_type1, error_code1])
train_for_two = np.hstack([error_type2, error_code2, quality_str])

test_for_one = np.hstack([error_type_test1, error_code_test1])
test_for_two = np.hstack([error_type_test2, error_code_test2, quality_str_test])

## 모델

In [33]:
# 모델 1번: lightgbm

def build_lightgbm(split_num, train, target, test, rnd):
    
    params = {
        'boosting_type' : 'gbdt',
        'objective'     : 'binary',
        'metric'        : 'auc',
        'seed'          : 1015
    }
    
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 1)), np.zeros((test.shape[0], 1))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        d_train= lgb.Dataset(X, y)
        d_val  = lgb.Dataset(valid_x, valid_y)
    
        #run traning
        model = lgb.train(
                            params,
                            train_set       = d_train,
                            num_boost_round = 1000,
                            valid_sets      = d_val,
                            verbose_eval    = 20, 
                            early_stopping_rounds = 10
                           )
 
        # save feat
        train_pred[val_idx] = model.predict(valid_x).reshape(-1,1)
        test_pred += model.predict(test).reshape(-1,1)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

# lgb_errtype_train1, lgb_errtype_test1 = build_lightgbm(5, error_type1, problem1, error_type_test1, 1)
# lgb_errtype_train2, lgb_errtype_test2 = build_lightgbm(5, error_type2, problem2, error_type_test2, 2)

# lgb_errcode_train1, lgb_errcode_test1 = build_lightgbm(5, error_code1, problem1, error_code_test1, 1)
# lgb_errcode_train2, lgb_errcode_test2 = build_lightgbm(5, error_code2, problem2, error_code_test2, 2)

# lgb_quality_train, lgb_quality_test = build_lightgbm(5, quality_str, problem2, quality_str_test, 1)

lgb_one_train1, lgb_one_test1 = build_lightgbm(5, train_for_one, problem1, test_for_one, 1)
lgb_one_train2, lgb_one_test2 = build_lightgbm(5, train_for_one, problem1, test_for_one, 2)

lgb_two_train1, lgb_two_test1 = build_lightgbm(5, train_for_two, problem2, test_for_two, 1)
lgb_two_train2, lgb_two_test2 = build_lightgbm(5, train_for_two, problem2, test_for_two, 2)

[LightGBM] [Info] Number of positive: 1466, number of negative: 3909
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001751 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5895
[LightGBM] [Info] Number of data points in the train set: 5375, number of used features: 98
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.272744 -> initscore=-0.980744
[LightGBM] [Info] Start training from score -0.980744
Training until validation scores don't improve for 10 rounds
[20]	valid_0's auc: 0.823417
[40]	valid_0's auc: 0.823406
Early stopping, best iteration is:
[36]	valid_0's auc: 0.824181
------------------
[LightGBM] [Info] Number of positive: 1466, number of negative: 3909
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001313 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5869
[LightGBM] [Info] Number of data poi

[40]	valid_0's auc: 0.800983
Early stopping, best iteration is:
[33]	valid_0's auc: 0.801366
------------------
[LightGBM] [Info] Number of positive: 2534, number of negative: 4091
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11433
[LightGBM] [Info] Number of data points in the train set: 6625, number of used features: 130
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.382491 -> initscore=-0.478990
[LightGBM] [Info] Start training from score -0.478990
Training until validation scores don't improve for 10 rounds
[20]	valid_0's auc: 0.784955
Early stopping, best iteration is:
[11]	valid_0's auc: 0.786116
------------------
[LightGBM] [Info] Number of positive: 2534, number of negative: 4091
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003715 seconds.
You can set `force_col_wise=true` to remove

In [34]:
# 모델 2번: Neural Network

def build_nn(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 1)), np.zeros((test.shape[0], 1))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        model = Sequential()
        model.add(Dense(units = 128, kernel_initializer = 'normal', input_dim = train.shape[1]))
        model.add(ReLU())
        model.add(Dropout(.3))
        model.add(Dense(units = 64, kernel_initializer = 'normal'))
        model.add(ReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.3))
        model.add(Dense(units = 32, kernel_initializer = 'normal'))
        model.add(ReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.3))
        model.add(Dense(units = 16, kernel_initializer = 'normal'))
        model.add(ReLU())
        model.add(BatchNormalization())
        model.add(Dropout(.3))
        model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam')
 
        print( 'Fitting neural network...' )
        model.fit(X, y,
                  validation_data = (valid_x, valid_y),
                  epochs=10
                 )

        # save feat
        train_pred[val_idx] = model.predict(valid_x)
        test_pred += model.predict(test)
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

# nn_errtype_train1, nn_errtype_test1 = build_nn(5, error_type, problem, error_type_test, 1)
# nn_errtype_train2, nn_errtype_test2 = build_nn(5, error_type, problem, error_type_test, 2)

# nn_errcode_train1, nn_errcode_test1 = build_nn(5, error_code, problem, error_code_test, 1)
# nn_errcode_train2, nn_errcode_test2 = build_nn(5, error_code, problem, error_code_test, 2)

# nn_quality_train1, nn_quality_test1 = build_nn(5, quality_str, problem, quality_str_test, 1)
# nn_quality_train2, nn_quality_test2 = build_nn(5, quality_str, problem, quality_str_test, 2)

nn_one_train1, nn_one_test1 = build_nn(5, train_for_one, problem1, test_for_one, 1)
nn_one_train2, nn_one_test2 = build_nn(5, train_for_one, problem1, test_for_one, 2)

nn_two_train1, nn_two_test1 = build_nn(5, train_for_two, problem2, test_for_two, 1)
nn_two_train2, nn_two_test2 = build_nn(5, train_for_two, problem2, test_for_two, 2)

In [35]:
# 모델 3번: catboost

def build_catboost(split_num, train, target, test, rnd):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((train.shape[0], 1)), np.zeros((test.shape[0], 1))

    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=233*rnd)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        model = cb.CatBoostClassifier(iterations=1500,
                                      learning_rate=0.01,
                                      l2_leaf_reg=3.5,
                                      depth=8,
                                      rsm=0.98,
                                      loss_function= 'Logloss',
                                      eval_metric='AUC',
                                      use_best_model=True,
                                      random_seed=42)

        model.fit(X, y,
                  eval_set=(valid_x, valid_y),
                  early_stopping_rounds=20)
        
        # save feat
        train_pred[val_idx] = model.predict(valid_x).reshape(-1,1)
        test_pred += model.predict(test).reshape(-1,1)/split_num
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred, test_pred

# catboost_errtype_train1, catboost_errtype_test1 = build_catboost(5, error_type, problem, error_type_test, 1)
# catboost_errtype_train2, catboost_errtype_test2 = build_catboost(5, error_type, problem, error_type_test, 2)

# catboost_errcode_train1, catboost_errcode_test1 = build_catboost(5, error_code, problem, error_code_test, 1)
# catboost_errcode_train2, catboost_errcode_test2 = build_catboost(5, error_code, problem, error_code_test, 2)

# catboost_quality_train1, catboost_quality_test1 = build_catboost(5, quality_str, problem, quality_str_test, 1)
# catboost_quality_train2, catboost_quality_test2 = build_catboost(5, quality_str, problem, quality_str_test, 2)

catboost_one_train1, catboost_one_test1 = build_catboost(5, train_for_one, problem1, test_for_one, 1)
catboost_one_train2, catboost_one_test2 = build_catboost(5, train_for_one, problem1, test_for_one, 2)

catboost_two_train1, catboost_two_test1 = build_catboost(5, train_for_two, problem2, test_for_two, 1)
catboost_two_train2, catboost_two_test2 = build_catboost(5, train_for_two, problem2, test_for_two, 2)

0:	test: 0.7630237	best: 0.7630237 (0)	total: 158ms	remaining: 3m 56s
1:	test: 0.7885536	best: 0.7885536 (1)	total: 164ms	remaining: 2m 2s
2:	test: 0.7904362	best: 0.7904362 (2)	total: 173ms	remaining: 1m 26s
3:	test: 0.7881283	best: 0.7904362 (2)	total: 179ms	remaining: 1m 6s
4:	test: 0.7904947	best: 0.7904947 (4)	total: 187ms	remaining: 55.9s
5:	test: 0.7940339	best: 0.7940339 (5)	total: 193ms	remaining: 48.1s
6:	test: 0.7950128	best: 0.7950128 (6)	total: 202ms	remaining: 43s
7:	test: 0.7927343	best: 0.7950128 (6)	total: 208ms	remaining: 38.8s
8:	test: 0.7917135	best: 0.7950128 (6)	total: 216ms	remaining: 35.8s
9:	test: 0.7929671	best: 0.7950128 (6)	total: 223ms	remaining: 33.2s
10:	test: 0.7961507	best: 0.7961507 (10)	total: 229ms	remaining: 31s
11:	test: 0.7970529	best: 0.7970529 (11)	total: 237ms	remaining: 29.4s
12:	test: 0.7995895	best: 0.7995895 (12)	total: 244ms	remaining: 27.9s
13:	test: 0.7989383	best: 0.7995895 (12)	total: 252ms	remaining: 26.7s
14:	test: 0.7984139	best: 0.

128:	test: 0.8139762	best: 0.8140599 (127)	total: 1.24s	remaining: 13.2s
129:	test: 0.8141882	best: 0.8141882 (129)	total: 1.26s	remaining: 13.3s
130:	test: 0.8143360	best: 0.8143360 (130)	total: 1.26s	remaining: 13.2s
131:	test: 0.8143611	best: 0.8143611 (131)	total: 1.27s	remaining: 13.2s
132:	test: 0.8143806	best: 0.8143806 (132)	total: 1.28s	remaining: 13.2s
133:	test: 0.8144782	best: 0.8144782 (133)	total: 1.29s	remaining: 13.2s
134:	test: 0.8147320	best: 0.8147320 (134)	total: 1.3s	remaining: 13.1s
135:	test: 0.8147766	best: 0.8147766 (135)	total: 1.3s	remaining: 13.1s
136:	test: 0.8147515	best: 0.8147766 (135)	total: 1.31s	remaining: 13.1s
137:	test: 0.8150165	best: 0.8150165 (137)	total: 1.32s	remaining: 13.1s
138:	test: 0.8150165	best: 0.8150165 (137)	total: 1.33s	remaining: 13s
139:	test: 0.8150221	best: 0.8150221 (139)	total: 1.34s	remaining: 13s
140:	test: 0.8151113	best: 0.8151113 (140)	total: 1.35s	remaining: 13s
141:	test: 0.8152536	best: 0.8152536 (141)	total: 1.35s	rem

246:	test: 0.8216500	best: 0.8216667 (244)	total: 2.31s	remaining: 11.7s
247:	test: 0.8217337	best: 0.8217337 (247)	total: 2.32s	remaining: 11.7s
248:	test: 0.8217114	best: 0.8217337 (247)	total: 2.33s	remaining: 11.7s
249:	test: 0.8218369	best: 0.8218369 (249)	total: 2.34s	remaining: 11.7s
250:	test: 0.8218201	best: 0.8218369 (249)	total: 2.34s	remaining: 11.7s
251:	test: 0.8219624	best: 0.8219624 (251)	total: 2.35s	remaining: 11.6s
252:	test: 0.8220488	best: 0.8220488 (252)	total: 2.36s	remaining: 11.6s
253:	test: 0.8221520	best: 0.8221520 (253)	total: 2.37s	remaining: 11.6s
254:	test: 0.8222719	best: 0.8222719 (254)	total: 2.38s	remaining: 11.6s
255:	test: 0.8222803	best: 0.8222803 (255)	total: 2.39s	remaining: 11.6s
256:	test: 0.8221687	best: 0.8222803 (255)	total: 2.4s	remaining: 11.6s
257:	test: 0.8220795	best: 0.8222803 (255)	total: 2.41s	remaining: 11.6s
258:	test: 0.8221827	best: 0.8222803 (255)	total: 2.42s	remaining: 11.6s
259:	test: 0.8221715	best: 0.8222803 (255)	total: 2.

362:	test: 0.8245672	best: 0.8245672 (362)	total: 3.39s	remaining: 10.6s
363:	test: 0.8245421	best: 0.8245672 (362)	total: 3.4s	remaining: 10.6s
364:	test: 0.8245812	best: 0.8245812 (364)	total: 3.4s	remaining: 10.6s
365:	test: 0.8245895	best: 0.8245895 (365)	total: 3.41s	remaining: 10.6s
366:	test: 0.8245951	best: 0.8245951 (366)	total: 3.42s	remaining: 10.6s
367:	test: 0.8245979	best: 0.8245979 (367)	total: 3.43s	remaining: 10.5s
368:	test: 0.8246146	best: 0.8246146 (368)	total: 3.43s	remaining: 10.5s
369:	test: 0.8245784	best: 0.8246146 (368)	total: 3.44s	remaining: 10.5s
370:	test: 0.8246174	best: 0.8246174 (370)	total: 3.45s	remaining: 10.5s
371:	test: 0.8245895	best: 0.8246174 (370)	total: 3.46s	remaining: 10.5s
372:	test: 0.8246342	best: 0.8246342 (372)	total: 3.46s	remaining: 10.5s
373:	test: 0.8245561	best: 0.8246342 (372)	total: 3.47s	remaining: 10.5s
374:	test: 0.8246648	best: 0.8246648 (374)	total: 3.48s	remaining: 10.4s
375:	test: 0.8246816	best: 0.8246816 (375)	total: 3.4

484:	test: 0.8266673	best: 0.8267119 (480)	total: 4.47s	remaining: 9.35s
485:	test: 0.8266478	best: 0.8267119 (480)	total: 4.47s	remaining: 9.34s
486:	test: 0.8266785	best: 0.8267119 (480)	total: 4.48s	remaining: 9.33s
487:	test: 0.8266785	best: 0.8267119 (480)	total: 4.49s	remaining: 9.31s
488:	test: 0.8267063	best: 0.8267119 (480)	total: 4.5s	remaining: 9.3s
489:	test: 0.8266729	best: 0.8267119 (480)	total: 4.51s	remaining: 9.29s
490:	test: 0.8267091	best: 0.8267119 (480)	total: 4.51s	remaining: 9.27s
491:	test: 0.8267565	best: 0.8267565 (491)	total: 4.52s	remaining: 9.27s
492:	test: 0.8267287	best: 0.8267565 (491)	total: 4.54s	remaining: 9.27s
493:	test: 0.8266980	best: 0.8267565 (491)	total: 4.54s	remaining: 9.26s
494:	test: 0.8266785	best: 0.8267565 (491)	total: 4.55s	remaining: 9.24s
495:	test: 0.8267259	best: 0.8267565 (491)	total: 4.57s	remaining: 9.24s
496:	test: 0.8267761	best: 0.8267761 (496)	total: 4.57s	remaining: 9.23s
497:	test: 0.8267677	best: 0.8267761 (496)	total: 4.5

...

117:	test: 0.7751340	best: 0.7751340 (117)	total: 1.88s	remaining: 22s
118:	test: 0.7751819	best: 0.7751819 (118)	total: 1.9s	remaining: 22s
119:	test: 0.7751865	best: 0.7751865 (119)	total: 1.91s	remaining: 22s
120:	test: 0.7753904	best: 0.7753904 (120)	total: 1.93s	remaining: 22s
121:	test: 0.7755139	best: 0.7755139 (121)	total: 1.94s	remaining: 21.9s
122:	test: 0.7758814	best: 0.7758814 (122)	total: 1.96s	remaining: 22s
123:	test: 0.7759880	best: 0.7759880 (123)	total: 1.97s	remaining: 21.9s
124:	test: 0.7761162	best: 0.7761162 (124)	total: 1.99s	remaining: 21.9s
125:	test: 0.7762134	best: 0.7762134 (125)	total: 2.01s	remaining: 21.9s
126:	test: 0.7763571	best: 0.7763571 (126)	total: 2.03s	remaining: 21.9s
127:	test: 0.7764343	best: 0.7764343 (127)	total: 2.05s	remaining: 22s
128:	test: 0.7763524	best: 0.7764343 (127)	total: 2.06s	remaining: 22s
129:	test: 0.7765702	best: 0.7765702 (129)	total: 2.08s	remaining: 21.9s
130:	test: 0.7766474	best: 0.7766474 (130)	total: 2.1s	remaining: 

230:	test: 0.7814840	best: 0.7815350 (229)	total: 3.73s	remaining: 20.5s
231:	test: 0.7815782	best: 0.7815782 (231)	total: 3.74s	remaining: 20.5s
232:	test: 0.7816600	best: 0.7816600 (232)	total: 3.77s	remaining: 20.5s
233:	test: 0.7816678	best: 0.7816678 (233)	total: 3.78s	remaining: 20.5s
234:	test: 0.7817079	best: 0.7817079 (234)	total: 3.79s	remaining: 20.4s
235:	test: 0.7818052	best: 0.7818052 (235)	total: 3.81s	remaining: 20.4s
236:	test: 0.7818840	best: 0.7818840 (236)	total: 3.82s	remaining: 20.4s
237:	test: 0.7819643	best: 0.7819643 (237)	total: 3.84s	remaining: 20.4s
238:	test: 0.7819565	best: 0.7819643 (237)	total: 3.85s	remaining: 20.3s
239:	test: 0.7820075	best: 0.7820075 (239)	total: 3.86s	remaining: 20.3s
240:	test: 0.7819859	best: 0.7820075 (239)	total: 3.88s	remaining: 20.3s
241:	test: 0.7820368	best: 0.7820368 (241)	total: 3.89s	remaining: 20.2s
242:	test: 0.7820230	best: 0.7820368 (241)	total: 3.91s	remaining: 20.2s
243:	test: 0.7820863	best: 0.7820863 (243)	total: 3

347:	test: 0.7846080	best: 0.7846080 (347)	total: 5.56s	remaining: 18.4s
348:	test: 0.7846281	best: 0.7846281 (348)	total: 5.58s	remaining: 18.4s
349:	test: 0.7846945	best: 0.7846945 (349)	total: 5.59s	remaining: 18.4s
350:	test: 0.7847223	best: 0.7847223 (350)	total: 5.6s	remaining: 18.3s
351:	test: 0.7848011	best: 0.7848011 (351)	total: 5.62s	remaining: 18.3s
352:	test: 0.7848103	best: 0.7848103 (352)	total: 5.63s	remaining: 18.3s
353:	test: 0.7849061	best: 0.7849061 (353)	total: 5.65s	remaining: 18.3s
354:	test: 0.7849076	best: 0.7849076 (354)	total: 5.66s	remaining: 18.3s
355:	test: 0.7849262	best: 0.7849262 (355)	total: 5.69s	remaining: 18.3s
356:	test: 0.7850327	best: 0.7850327 (356)	total: 5.71s	remaining: 18.3s
357:	test: 0.7851655	best: 0.7851655 (357)	total: 5.73s	remaining: 18.3s
358:	test: 0.7851856	best: 0.7851856 (358)	total: 5.74s	remaining: 18.3s
359:	test: 0.7852396	best: 0.7852396 (359)	total: 5.76s	remaining: 18.2s
360:	test: 0.7852458	best: 0.7852458 (360)	total: 5.

466:	test: 0.7872287	best: 0.7872287 (466)	total: 7.38s	remaining: 16.3s
467:	test: 0.7872287	best: 0.7872287 (466)	total: 7.39s	remaining: 16.3s
468:	test: 0.7872503	best: 0.7872503 (468)	total: 7.41s	remaining: 16.3s
469:	test: 0.7873043	best: 0.7873043 (469)	total: 7.42s	remaining: 16.3s
470:	test: 0.7872997	best: 0.7873043 (469)	total: 7.44s	remaining: 16.3s
471:	test: 0.7872951	best: 0.7873043 (469)	total: 7.45s	remaining: 16.2s
472:	test: 0.7873151	best: 0.7873151 (472)	total: 7.47s	remaining: 16.2s
473:	test: 0.7872966	best: 0.7873151 (472)	total: 7.48s	remaining: 16.2s
474:	test: 0.7873908	best: 0.7873908 (474)	total: 7.49s	remaining: 16.2s
475:	test: 0.7874402	best: 0.7874402 (475)	total: 7.51s	remaining: 16.1s
476:	test: 0.7874618	best: 0.7874618 (476)	total: 7.52s	remaining: 16.1s
477:	test: 0.7874418	best: 0.7874618 (476)	total: 7.53s	remaining: 16.1s
478:	test: 0.7874526	best: 0.7874618 (476)	total: 7.55s	remaining: 16.1s
479:	test: 0.7875221	best: 0.7875221 (479)	total: 7

------------------
0:	test: 0.7459950	best: 0.7459950 (0)	total: 12.7ms	remaining: 19.1s
1:	test: 0.7583872	best: 0.7583872 (1)	total: 24.8ms	remaining: 18.6s
2:	test: 0.7563400	best: 0.7583872 (1)	total: 40.9ms	remaining: 20.4s
3:	test: 0.7589560	best: 0.7589560 (3)	total: 55ms	remaining: 20.6s
4:	test: 0.7624331	best: 0.7624331 (4)	total: 72.3ms	remaining: 21.6s
5:	test: 0.7659943	best: 0.7659943 (5)	total: 86.2ms	remaining: 21.5s
6:	test: 0.7707764	best: 0.7707764 (6)	total: 115ms	remaining: 24.6s
7:	test: 0.7713358	best: 0.7713358 (7)	total: 127ms	remaining: 23.7s
8:	test: 0.7733159	best: 0.7733159 (8)	total: 144ms	remaining: 23.8s
9:	test: 0.7763007	best: 0.7763007 (9)	total: 156ms	remaining: 23.2s
10:	test: 0.7767251	best: 0.7767251 (10)	total: 172ms	remaining: 23.3s
11:	test: 0.7768471	best: 0.7768471 (11)	total: 184ms	remaining: 22.9s
12:	test: 0.7772576	best: 0.7772576 (12)	total: 200ms	remaining: 22.9s
13:	test: 0.7772591	best: 0.7772591 (13)	total: 212ms	remaining: 22.5s
14:

## stacking ensemble

In [21]:
# train = np.hstack([lgb_errtype_train1, lgb_errtype_train2,
#                    lgb_errcode_train1, lgb_errcode_train2,
#                    lgb_quality_train1, lgb_quality_train2,
# #                    nn_errtype_train1, nn_errtype_train2,
# #                    nn_errcode_train1, nn_errcode_train2,
# #                    nn_quality_train1, nn_quality_train2,
#                    catboost_errtype_train1, catboost_errtype_train2,
#                    catboost_errcode_train1, catboost_errcode_train2,
#                    catboost_quality_train1, catboost_quality_train2
#                   ])

# test = np.hstack([lgb_errtype_test1, lgb_errtype_test2,
#                   lgb_errcode_test1, lgb_errcode_test2,
#                   lgb_quality_test1, lgb_quality_test2,
# #                   nn_errtype_test1, nn_errtype_test2,
# #                   nn_errcode_test1, nn_errcode_test2,
# #                   nn_quality_test1, nn_quality_test2,
#                   catboost_errtype_test1, catboost_errtype_test2,
#                   catboost_errcode_test1, catboost_errcode_test2,
#                   catboost_quality_test1, catboost_quality_test2
#                  ])

# print(train.shape)
# print(test.shape)

(15000, 12)
(14999, 12)


In [36]:
train_one = np.hstack([lgb_one_train1, lgb_one_train2,
                   catboost_one_train1, catboost_one_train2
                  ])

test_one = np.hstack([lgb_one_test1, lgb_one_test2,
                  catboost_one_test1, catboost_one_test2
                 ])

train_two = np.hstack([lgb_two_train1, lgb_two_train2,
                   catboost_two_train1, catboost_two_train2
                  ])

test_two = np.hstack([lgb_two_test1, lgb_two_test2,
                  catboost_two_test1, catboost_two_test2
                 ])

print(train_one.shape)
print(train_two.shape)
print(test_one.shape)
print(test_two.shape)

(6719, 4)
(8281, 4)
(6731, 4)
(8267, 4)


In [37]:
# 최종 앙상블

def ensemble_xgboost(split_num, train, target, test):

    test_pred, test_pred_proba = np.zeros((test.shape[0], 1)), np.zeros((test.shape[0], 1))
    
    skf = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=2021)
    for train_idx, val_idx in skf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        model = xgb.XGBClassifier(
            learning_rate =0.1,
            n_estimators=1000,
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            eval_metric='logloss',
            nthread=4,
            scale_pos_weight=1,
            seed=27)

        model.fit(X, y,
                  eval_set=[(valid_x, valid_y)],
                  early_stopping_rounds=10)
        
        # save feat
        test_pred_proba += model.predict_proba(test)[:,1].reshape(-1,1)/split_num
        
        
        # release
        del model
        gc.collect()
        print('------------------')
    
#     sample_submssion['problem'] = test_pred_proba
#     sample_submssion.to_csv("stacking_ensemble_proba.csv", index = False)
    
#     print(sample_submssion)
    return test_pred_proba

In [38]:
test_pred_proba1 = ensemble_xgboost(5, train_one, problem1, test_one)
test_pred_proba2 = ensemble_xgboost(5, train_two, problem2, test_two)

[0]	validation_0-logloss:0.65065
[1]	validation_0-logloss:0.61574
[2]	validation_0-logloss:0.58738
[3]	validation_0-logloss:0.56309
[4]	validation_0-logloss:0.54263
[5]	validation_0-logloss:0.52568
[6]	validation_0-logloss:0.51156
[7]	validation_0-logloss:0.49992
[8]	validation_0-logloss:0.49032
[9]	validation_0-logloss:0.48186
[10]	validation_0-logloss:0.47462
[11]	validation_0-logloss:0.46843
[12]	validation_0-logloss:0.46340
[13]	validation_0-logloss:0.45850
[14]	validation_0-logloss:0.45470
[15]	validation_0-logloss:0.45191
[16]	validation_0-logloss:0.44882
[17]	validation_0-logloss:0.44646
[18]	validation_0-logloss:0.44471
[19]	validation_0-logloss:0.44326
[20]	validation_0-logloss:0.44180
[21]	validation_0-logloss:0.44087
[22]	validation_0-logloss:0.44015
[23]	validation_0-logloss:0.43934
[24]	validation_0-logloss:0.43888
[25]	validation_0-logloss:0.43799
[26]	validation_0-logloss:0.43784
[27]	validation_0-logloss:0.43747
[28]	validation_0-logloss:0.43713
[29]	validation_0-loglos

[13]	validation_0-logloss:0.53468
[14]	validation_0-logloss:0.53283
[15]	validation_0-logloss:0.53204
[16]	validation_0-logloss:0.53054
[17]	validation_0-logloss:0.52968
[18]	validation_0-logloss:0.52879
[19]	validation_0-logloss:0.52786
[20]	validation_0-logloss:0.52733
[21]	validation_0-logloss:0.52669
[22]	validation_0-logloss:0.52659
[23]	validation_0-logloss:0.52640
[24]	validation_0-logloss:0.52645
[25]	validation_0-logloss:0.52656
[26]	validation_0-logloss:0.52608
[27]	validation_0-logloss:0.52623
[28]	validation_0-logloss:0.52616
[29]	validation_0-logloss:0.52612
[30]	validation_0-logloss:0.52604
[31]	validation_0-logloss:0.52622
[32]	validation_0-logloss:0.52667
[33]	validation_0-logloss:0.52674
[34]	validation_0-logloss:0.52680
[35]	validation_0-logloss:0.52696
[36]	validation_0-logloss:0.52703
[37]	validation_0-logloss:0.52682
[38]	validation_0-logloss:0.52725
[39]	validation_0-logloss:0.52713
------------------
[0]	validation_0-logloss:0.66134
[1]	validation_0-logloss:0.635

In [ ]:
test_err1['pred'] = test_pred_proba1
test_err2['pred'] = test_pred_proba2

In [89]:
result1 = pd.DataFrame(data=test_pred_proba1, columns=['problem'])
result1.index = test_err1['user_id'].unique()

result2 = pd.DataFrame(data=test_pred_proba2, columns=['problem'])
result2.index = test_err2['user_id'].unique()

result = pd.concat([result1, result2])
result = result.sort_index(ascending=True)
result

,problem
30000,0.794677
30001,0.212551
30002,0.690209
30003,0.774653
30004,0.583904
...,...
44994,0.218688
44995,0.229100
44996,0.667409
44997,0.824156


In [93]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion.index = sample_submssion['user_id']
sample_submssion['problem'] = result['problem']
sample_submssion = sample_submssion.reset_index(drop=True)
sample_submssion.to_csv("ensemble.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.794677
1,30001,0.212551
2,30002,0.690209
3,30003,0.774653
4,30004,0.583904
...,...,...
14994,44994,0.218688
14995,44995,0.229100
14996,44996,0.667409
14997,44997,0.824156
